In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting_for_ccle

import setting_for_tcga

In [ ]:
path_for_ccle = path(setting_for_ccle)

path_for_tcga = path(setting_for_tcga)

In [ ]:
feature_x_sample_for_ccle = pd.read_table(
    path_for_ccle['feature_x_sample_file_path'],
    index_col=0,
)

print(feature_x_sample_for_ccle.shape)

feature_x_sample_for_tcga = pd.read_table(
    path_for_tcga['feature_x_sample_file_path'],
    index_col=0,
)

print(feature_x_sample_for_tcga.shape)

In [ ]:
feature_skew_t_pdf_fit_parameter_for_ccle = pd.read_table(
    path_for_ccle['feature_skew_t_pdf_fit_parameter_file_path'],
    index_col=0,
)

print(feature_skew_t_pdf_fit_parameter_for_ccle.shape)

feature_skew_t_pdf_fit_parameter_for_tcga = pd.read_table(
    path_for_tcga['feature_skew_t_pdf_fit_parameter_file_path'],
    index_col=0,
)

print(feature_skew_t_pdf_fit_parameter_for_tcga.shape)

In [ ]:
from numpy import finfo, linspace, log
from statsmodels.sandbox.distributions.extras import ACSkewT_gen

skew_t_model = ACSkewT_gen()

eps = finfo(float).eps

def compute_kld(
    feature,
    feature_x_sample_for_0,
    feature_x_sample_for_1,
    feature_skew_t_pdf_fit_parameter_for_0,
    feature_skew_t_pdf_fit_parameter_for_1,
    n_grid=64,
    plot=False,
    name_0='Data 0',
    name_1='Data 1',
):
    
    feature_values_for_0 = feature_x_sample_for_0.loc[feature]

    feature_values_for_1 = feature_x_sample_for_1.loc[feature]

    min_ = min(
        feature_values_for_0.min(),
        feature_values_for_1.min(),
    )

    max_ = max(
        feature_values_for_0.max(),
        feature_values_for_1.max(),
    )

    grid = linspace(
        min_,
        max_,
        n_grid,
    )

    n, location_for_0, scale_for_0, degree_of_freedom_for_0, shape_for_0 = feature_skew_t_pdf_fit_parameter_for_0.loc[feature]

    pdf_for_0 = skew_t_model.pdf(
        grid,
        degree_of_freedom_for_0,
        shape_for_0,
        loc=location_for_0,
        scale=scale_for_0,
    )
    
    n, location_for_1, scale_for_1, degree_of_freedom_for_1, shape_for_1 = feature_skew_t_pdf_fit_parameter_for_1.loc[feature]

    pdf_for_1 = skew_t_model.pdf(
        grid,
        degree_of_freedom_for_1,
        shape_for_1,
        loc=location_for_1,
        scale=scale_for_1,
    )
    
    pdf_for_0[pdf_for_0 < eps] = eps
    
    pdf_for_1[pdf_for_1 < eps] = eps
    
    kld = pdf_for_0 * log(pdf_for_0 / pdf_for_1)
    
    kld_sum = kld.sum() / n_grid
    
    if plot:
        
        ccal.plot_points(
            (
                name_0,
                name_1,
            ),
            (grid, ) * 2,
            (
                pdf_for_0,
                pdf_for_1,
            ),
            title='{}<br>{:.3f}'.format(
                feature,
                kld_sum,
            ),
        )

    return kld_sum

In [ ]:
common_features = feature_x_sample_for_ccle.index & feature_x_sample_for_tcga.index

feature_kld = [compute_kld(
        feature,
        feature_x_sample_for_ccle,
        feature_x_sample_for_tcga,
        feature_skew_t_pdf_fit_parameter_for_ccle,
        feature_skew_t_pdf_fit_parameter_for_tcga,
    ) for feature in common_features]
    
feature_kld = pd.Series(
    feature_kld,
    name='KLD',
    index=common_features,
).sort_values()

feature_kld.index.name = 'Feature'

ccal.plot_points(
    ('KLD', ),
    (tuple(range(feature_kld.size)), ),
    (feature_kld, ),
    texts=(feature_kld.index, ),
    title='Feature KLD',
    xaxis_title='Rank',
    yaxis_title='KLD',
)

In [ ]:
feature_kld = feature_kld[feature_kld < 0.5]

feature_kld.size

In [ ]:
n_extreme = 3

for i in list(range(n_extreme)) + list(range(
    -n_extreme,
    0,
)):
    
    feature = feature_kld.index[i]
    
    print(feature)
    
    kld = feature_kld[i]

    compute_kld(
        feature,
        feature_x_sample_for_ccle,
        feature_x_sample_for_tcga,
        feature_skew_t_pdf_fit_parameter_for_ccle,
        feature_skew_t_pdf_fit_parameter_for_tcga,
        plot=True,
        name_0='CCLE',
        name_1='TCGA',
    )

In [ ]:
feature_kld.index.to_series().to_csv(
    '../output/features.txt',
    index=False,
)